In [1]:
import cv2 as cv
import mediapipe as mp
import time
import serial as s
import numpy as np

# conf Serial
#hw = s.Serial(port='COM6' ,baudrate=115200, timeout=1, write_timeout=1)

class handDetector():
    def __init__(self,mode=False, maxHands=2,detecConf=0.5, trackConf=0.5):
        self.mode =mode
        self.maxHands = maxHands
        self.detecConf = detecConf
        self.trackConf = trackConf
        self.mpHands =mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detecConf, self.trackConf)
        self.mpDraw =mp.solutions.drawing_utils

    def findHands(self, img, draw = True):
        imgRGB = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks: #leo los puntos de las manos
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms) #self.mpHands.HAND_CONNECTIONS)
   
        return img
    
    def DrawPoints(self, blank, handNum=0, draw=True):
        if self.results.multi_hand_landmarks:
            Myhand = self.results.multi_hand_landmarks[handNum]
            for id, lm in enumerate(Myhand.landmark): #enumera y saca el dato
                        h, w = blank.shape
                        cx, cy = int(lm.x*w), int(lm.y*h) # coordenada de cada punto
                        #print(id, cx, cy)
                        #lmList.append([id,cx,cy])
                        if draw:
                            #if id == 8:
                                cv.circle(blank, (cx, cy), 3, (255,0,0), -1)
                                
        return blank
               
    
    def findPosition(self, img, handNum=0, draw=True):
        lmList = []
        if self.results.multi_hand_landmarks:
            Myhand = self.results.multi_hand_landmarks[handNum]
            for id, lm in enumerate(Myhand.landmark): #enumera y saca el dato
                        h, w, c = img.shape
                        cx, cy = int(lm.x*w), int(lm.y*h) # coordenada de cada punto
                        #print(id, cx, cy)
                        lmList.append([id,cx,cy])
                        if draw:
                            if id == 8:
                                cv.circle(img, (cx, cy), 15, (0,0,0), -1)
                                
        return lmList
        
def main():
    cap = cv.VideoCapture(0)
    pTime=0 #tiempo previo
    cTime=0
    detector = handDetector()
    
    while True:
        isTrue, img1 = cap.read()
        if img1.any() != None:
            
            img =cv.flip(img1, 1)
            blank=np.zeros((img.shape[0],img.shape[1]), dtype='uint8')
            img= detector.findHands(img)
            lmList = detector.findPosition(img)
            blank = detector.DrawPoints(blank)
            if len(lmList) !=  0:
#                 print(lmList[8], type(lmList[8]))
                p_indice = lmList[8]
                p_pulgar = lmList[4]
                p_mayor = lmList[12]
                p_anular = lmList[16]
                p_menique = lmList[20]
#                 cc = p[1:2]
#                 print(cc)
                cv.circle(blank, (p_indice[1],p_indice[2]), 6, (255,255,255), -1)
                cv.circle(blank, (p_pulgar[1],p_pulgar[2]), 6, (255,255,255), -1)
                cv.circle(blank, (p_mayor[1],p_mayor[2]), 6, (255,255,255), -1)
                cv.circle(blank, (p_anular[1],p_anular[2]), 6, (255,255,255), -1)
                cv.circle(blank, (p_menique[1],p_menique[2]), 6, (255,255,255), -1)
                cv.line(blank, (p_indice[1],p_indice[2]),(p_pulgar[1],p_pulgar[2]), (50,0,50), 3 )
                
                #hw.write(str(cc).encode('utf-8'))
            
            cTime = time.time()
            fps = 1/(cTime-pTime)
            pTime = cTime
            cv.putText(img, 'fps: ' + str(int(fps)), (10,30), cv.FONT_HERSHEY_PLAIN, 1, (0,0,0), 1)
            cv.putText(blank, 'fps: ' + str(int(fps)), (10,30), cv.FONT_HERSHEY_PLAIN, 1, (255,255,255), 1)
            cv.imshow('video',img)
            cv.imshow('esq',blank)
            
            if cv.waitKey(20) & 0xFF==ord('d'):
                 break
        if img1.any() == None:
            print('Camara no detectada ')
            break

    cap.release()
    cv.destroyAllWindows()
    #hw.close()



if __name__ == "__main__":
    main()

AttributeError: 'NoneType' object has no attribute 'any'

In [9]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.
